In [7]:
#Ex.1
import pandas as pd

# Загрузка датасета
df = pd.read_csv('train.csv')

# Смотрим размер таблицы до удаления дубликатов
print("Количество строк ДО:", df.shape[0])

# Проверим, есть ли полные дубликаты строк
duplicates = df.duplicated()
print("Количество дубликатов:", duplicates.sum())

# Удаляем дубликаты
df_cleaned = df.drop_duplicates()

# Проверим размер таблицы после удаления
print("Количество строк ПОСЛЕ:", df_cleaned.shape[0])

print(f"Удалено строк: {df.shape[0] - df_cleaned.shape[0]}")

Количество строк ДО: 891
Количество дубликатов: 0
Количество строк ПОСЛЕ: 891
Удалено строк: 0


In [12]:
#Ex.2
import pandas as pd
from sklearn.impute import SimpleImputer

# 1. Загрузка данных
df = pd.read_csv('train.csv')

# 2. Проверка пропущенных значений
print("🔍 Пропущенные значения до обработки:")
print(df.isnull().sum())
print("-" * 40)

# 3. Удаление строк с пропущенными Age (создаём копию)
df_dropped_age = df.dropna(subset=['Age'])
print(f"📉 Размер после удаления строк с пустым Age: {df_dropped_age.shape}")

# 4. Заполнение Age средним значением
df_age_filled = df.copy()
df_age_filled['Age'] = df_age_filled['Age'].fillna(df_age_filled['Age'].mean())

# 5. Заполнение Embarked значением "Unknown"
df_age_filled['Embarked'] = df_age_filled['Embarked'].fillna('Unknown')

# 6. Альтернатива: используем SimpleImputer для Age
imputer = SimpleImputer(strategy='mean')
df_age_filled['Age'] = imputer.fit_transform(df_age_filled[['Age']])

# 7. Проверка после обработки
print("\n✅ Пропущенные значения после обработки:")
print(df_age_filled.isnull().sum())

# 8. Вывод первых строк обновлённого DataFrame
print("\n👀 Первые строки обработанного DataFrame:")
print(df_age_filled.head())


🔍 Пропущенные значения до обработки:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------------------------------------
📉 Размер после удаления строк с пустым Age: (714, 12)

✅ Пропущенные значения после обработки:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

👀 Первые строки обработанного DataFrame:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0        

In [14]:
#Ex.3
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 1. Загрузка датасета
df = pd.read_csv('train.csv')

# 2. Создание признака "FamilySize"
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# 3. Извлечение "Title" из "Name"
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

# Можно объединить редкие титулы:
df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 
                                   'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

# 4. Кодирование категориальных признаков

# Label encoding for 'Sex'
le_sex = LabelEncoder()
df['Sex_encoded'] = le_sex.fit_transform(df['Sex'])

# One-hot encoding for 'Embarked' and 'Title'
df = pd.get_dummies(df, columns=['Embarked', 'Title'], prefix=['Embarked', 'Title'])

# 5. Нормализация числовых признаков
scaler = MinMaxScaler()
df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])

# 6. Проверим результат
print(df[['Sex', 'Sex_encoded', 'FamilySize'] + [col for col in df.columns if col.startswith('Embarked_') or col.startswith('Title_')]].head())


      Sex  Sex_encoded  FamilySize  Embarked_C  Embarked_Q  Embarked_S  \
0    male            1           2       False       False        True   
1  female            0           2        True       False       False   
2  female            0           1       False       False        True   
3  female            0           2       False       False        True   
4    male            1           1       False       False        True   

   Title_Master  Title_Miss  Title_Mr  Title_Mrs  Title_Rare  
0         False       False      True      False       False  
1         False       False     False       True       False  
2         False        True     False      False       False  
3         False       False     False       True       False  
4         False       False      True      False       False  


In [17]:
import pandas as pd
import numpy as np
from scipy import stats

# Загрузка данных
df = pd.read_csv("train.csv")

# --- I. Обнаружение выбросов с помощью IQR ---
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

# Найдём выбросы
outliers_fare = detect_outliers_iqr(df, 'Fare')
outliers_age = detect_outliers_iqr(df, 'Age')

print(f"🎯 IQR: Количество выбросов в 'Fare': {len(outliers_fare)}")
print(f"🎯 IQR: Количество выбросов в 'Age': {len(outliers_age)}")

# --- II. Альтернатива: Z-оценка (исправленная версия) ---
# Удалим строки с NaN в нужных столбцах
df_no_na = df.dropna(subset=['Fare', 'Age'])

# Вычислим Z-оценку
z_scores = np.abs(stats.zscore(df_no_na[['Fare', 'Age']]))
mask = (z_scores < 3).all(axis=1)

# Применим фильтр
df_z_filtered = df_no_na[mask]
print(f"\n✅ Размер данных после удаления выбросов по Z-score: {df_z_filtered.shape}")

# --- III. Кэппинг (ограничение выбросов) ---
def cap_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[column] = np.where(data[column] > upper_bound, upper_bound,
                            np.where(data[column] < lower_bound, lower_bound, data[column]))
    return data

df_capped = df.copy()
df_capped = cap_outliers(df_capped, 'Fare')
df_capped = cap_outliers(df_capped, 'Age')

print("\n👀 После кэппинга:")
print("Макс Fare:", df_capped['Fare'].max())
print("Мин Fare:", df_capped['Fare'].min())
print("Макс Age:", df_capped['Age'].max())
print("Мин Age:", df_capped['Age'].min())


🎯 IQR: Количество выбросов в 'Fare': 116
🎯 IQR: Количество выбросов в 'Age': 11

✅ Размер данных после удаления выбросов по Z-score: (694, 12)

👀 После кэппинга:
Макс Fare: 65.6344
Мин Fare: 0.0
Макс Age: 64.8125
Мин Age: 0.42


In [19]:
#Ex.5
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Загрузка данных
df = pd.read_csv("train.csv")

# Оставим только числовые столбцы и удалим строки с пропущенными значениями
numeric_cols = ['Age', 'Fare', 'SibSp', 'Parch']
df_numeric = df[numeric_cols].dropna()

# --- I. Стандартизация (Standardization) ---
# Приводит данные к распределению со средним = 0 и стандартным отклонением = 1
scaler_std = StandardScaler()
standardized_data = scaler_std.fit_transform(df_numeric)

df_standardized = pd.DataFrame(standardized_data, columns=[col + '_std' for col in df_numeric.columns])

# --- II. Нормализация (Min-Max Normalization) ---
# Приводит данные в диапазон [0, 1]
scaler_norm = MinMaxScaler()
normalized_data = scaler_norm.fit_transform(df_numeric)

df_normalized = pd.DataFrame(normalized_data, columns=[col + '_norm' for col in df_numeric.columns])

# --- III. Объединение результатов для наглядности ---
df_result = pd.concat([df_numeric.reset_index(drop=True), df_standardized, df_normalized], axis=1)

# Показать первые 5 строк
print(df_result.head())


    Age     Fare  SibSp  Parch   Age_std  Fare_std  SibSp_std  Parch_std  \
0  22.0   7.2500      1      0 -0.530377 -0.518978   0.524570  -0.505895   
1  38.0  71.2833      1      0  0.571831  0.691897   0.524570  -0.505895   
2  26.0   7.9250      0      0 -0.254825 -0.506214  -0.551703  -0.505895   
3  35.0  53.1000      1      0  0.365167  0.348049   0.524570  -0.505895   
4  35.0   8.0500      0      0  0.365167 -0.503850  -0.551703  -0.505895   

   Age_norm  Fare_norm  SibSp_norm  Parch_norm  
0  0.271174   0.014151         0.2         0.0  
1  0.472229   0.139136         0.2         0.0  
2  0.321438   0.015469         0.0         0.0  
3  0.434531   0.103644         0.2         0.0  
4  0.434531   0.015713         0.0         0.0  


In [20]:
#Ex.6
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
df = pd.read_csv("train.csv")

# --- I. Определим категориальные переменные ---
categorical_cols = ['Sex', 'Embarked', 'Pclass']  # Pclass можно трактовать как порядковую переменную

# --- II. One-hot encoding для номинативных переменных ---
# 'Sex' и 'Embarked' — это номинативные признаки
df_encoded = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)

# --- III. Label encoding для порядковой переменной ---
# 'Pclass' — имеет логический порядок (1 > 2 > 3)
label_encoder = LabelEncoder()
df_encoded['Pclass_encoded'] = label_encoder.fit_transform(df['Pclass'])

# (Необязательно) Удалим оригинальный 'Pclass' если нужен только числовой
df_encoded.drop(columns=['Pclass'], inplace=True)

# --- IV. Результат ---
print(df_encoded[['Sex_male', 'Embarked_Q', 'Embarked_S', 'Pclass_encoded']].head())


   Sex_male  Embarked_Q  Embarked_S  Pclass_encoded
0      True       False        True               2
1     False       False       False               0
2     False       False        True               2
3     False       False        True               0
4      True       False        True               2


In [21]:
#Ex.7
import pandas as pd

# Загрузка данных
df = pd.read_csv("train.csv")

# --- I. Удалим строки с пропущенным возрастом ---
df = df.dropna(subset=['Age'])

# --- II. Создадим возрастные категории (биннинг) ---
# Например: 0–12: дети, 13–19: подростки, 20–35: молодые взрослые, 36–60: взрослые, 61+: пожилые
age_bins = [0, 12, 19, 35, 60, 100]
age_labels = ['Child', 'Teen', 'Young Adult', 'Adult', 'Senior']

df['AgeGroup'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels)

# --- III. Применим one-hot encoding к возрастным группам ---
df_age_dummies = pd.get_dummies(df['AgeGroup'], prefix='AgeGroup')

# --- IV. Объединим результат с оригинальным DataFrame ---
df = pd.concat([df, df_age_dummies], axis=1)

# --- V. Просмотр результата ---
print(df[['Age', 'AgeGroup'] + list(df_age_dummies.columns)].head())


    Age     AgeGroup  AgeGroup_Child  AgeGroup_Teen  AgeGroup_Young Adult  \
0  22.0  Young Adult           False          False                  True   
1  38.0        Adult           False          False                 False   
2  26.0  Young Adult           False          False                  True   
3  35.0  Young Adult           False          False                  True   
4  35.0  Young Adult           False          False                  True   

   AgeGroup_Adult  AgeGroup_Senior  
0           False            False  
1            True            False  
2           False            False  
3           False            False  
4           False            False  
